In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import json

In [33]:
#load previous result
folder_name = "results/job_match_0303_1550"

with open(folder_name + '/test_dict.json') as f:
    test_dict = json.load(f)

#get related occupation, filtered by primary-short (most relevant)
related = pd.read_excel('datasets/related_occupations.xlsx').astype(str)
related.columns = related.columns.str.lower().str.replace(" ","_").str.replace("o*net-soc_", "")
related = related[related["relatedness_tier"] == "Primary-Short"]


def get_result(code):
    with open(folder_name + '/' + code + '.json') as f:
        career = pd.read_json(f)

    return career

#match suggested career to the related career
def match(code, career):
    #check if direct match
    career = career[career["fit"] == "Best"]
    if len(career[career["code"] == code]) == 1: #direct match
        return 1
    else: #check related match
        related_oc = related[related["code"] == code].drop(labels=["code", "title", "relatedness_tier", "index"], axis=1)
        related_oc.columns = related_oc.columns.str.replace("related_", "")
        merged_df = related_oc.merge(right=career, on="code")
        related_match = len(merged_df) / 5
        return related_match

In [34]:
score = 0
for code in test_dict.values():
    score += match(code, get_result(code))
score / len(test_dict)

0.44000000000000006